<a href="https://colab.research.google.com/github/PGestrich/AML-Time-Series-Forecasting/blob/main/AML_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AML Project - Time Series Forecasting

Julian Nürk, 

Prisca Gestrich

## Data Stuff

### 1. Utility / Loading Data

In [1]:
#general
import numpy as np
import pandas as pd
!pip install -q kaggle


from google.colab import files

In [2]:
files.upload()
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c 'store-sales-time-series-forecasting'
! mkdir data
! unzip store-sales-time-series-forecasting.zip -d data

Saving kaggle.json to kaggle.json
 42% 9.00M/21.4M [00:00<00:00, 76.7MB/s]
100% 21.4M/21.4M [00:00<00:00, 117MB/s] 
Archive:  store-sales-time-series-forecasting.zip
  inflating: data/holidays_events.csv  
  inflating: data/oil.csv            
  inflating: data/sample_submission.csv  
  inflating: data/stores.csv         
  inflating: data/test.csv           
  inflating: data/train.csv          
  inflating: data/transactions.csv   


In [3]:
oil = pd.read_csv('/content/data/oil.csv')
holidays = pd.read_csv('/content/data/holidays_events.csv')
stores = pd.read_csv('/content/data/stores.csv')
train = pd.read_csv('/content/data/train.csv')
transactions = pd.read_csv('/content/data/transactions.csv')
test = pd.read_csv('/content/data/test.csv')



## Process Data

holiday handling:

0: work day

1 - weekend / bridge day

2 - holiday

type additional: ??? 

transferred holidays are considered normal days

only first holiday in holidays is considered




In [ ]:
#change dtype of date column to datetime
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])
train['date']=pd.to_datetime(train ['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
test['date'] = pd.to_datetime(test['date'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

#expand oil to include all dates + interpolate missing data
oil = oil.set_index('date').asfreq('D').reset_index()
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate('linear').ffill().bfill()
train = train.merge(oil)
train = train.rename(columns={"dcoilwtico": "oilprice"})


#change family names & type to numeric values
encoder_family = LabelEncoder()
train['family_id']=encoder_family.fit_transform(train['family'])



#split up date into multiple informations
train['day'] = train['date'].apply(lambda time: time.day)
train['month'] = train['date'].apply(lambda time: time.month)
train['weekday'] = train['date'].apply(lambda time: time.dayofweek)
train['year'] = train['date'].apply(lambda time: time.year)

#remove noise - 3 months
to_drop = train.loc[train['date'].between('2016-04-16', '2016-07-16')]
train = train.drop(to_drop.index)

In [ ]:
train

,id,date,store_nbr,family,sales,onpromotion,oilprice,family_id,day,month,weekday,year
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14,0,1,1,1,2013
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14,1,1,1,1,2013
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14,2,1,1,1,2013
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14,3,1,1,1,2013
4,4,2013-01-01,1,BOOKS,0.000,0,93.14,4,1,1,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,28,15,8,1,2017
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,29,15,8,1,2017
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,30,15,8,1,2017
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,31,15,8,1,2017


prob. faster: do holiday lookup table with clusters instead of store numbers

In [ ]:
#holiday handling
def isholiday(row):
  #data
  date = row['date']
  event = holidays.loc[holidays['date'] == date][0:1]
  id = row['store_nbr']
  city = stores.loc[stores['store_nbr'] == id]['city'].values
  state = stores.loc[stores['store_nbr'] == id]['state'].values
  
  
  #check if events apply:
  if len(event) > 0:
    national = event['locale'].values == 'National'
    regional = event['locale'].values == 'Regional' and event['locale_name'].values == state
    local = event['locale'].values == 'Local' and event['locale_name'].values == city
    
    if national[0] or regional[0] or local[0]:
      if event['type'].values == 'Holiday' and  event['transferred'].values == False:
        return 2
      elif event['type'].values == 'Transfer':
        return 2
      elif event['type'].values == 'Bridge':
        return 1
      elif event['type'].values == 'Work Day':
        return 0

  #otherwise: check if weekend
  if row['weekday']< 5:
    return 0
  else: 
    return 1


#lookup table  (adds dates to stores table and gets holiday type)
date1, date2 = train['date'].min(), train['date'].max()
holiday_lookup = stores[['store_nbr', 'type', 'cluster']].copy()
holiday_lookup.loc[:,'holiday'] = 0

holiday_lookup = pd.merge(holiday_lookup, pd.DataFrame({'date': pd.date_range(date1, date2, freq = 'd')}), how = "cross")
holiday_lookup['weekday'] = holiday_lookup['date'].apply(lambda time: time.dayofweek)
holiday_lookup.loc[:, 'holiday'] = holiday_lookup.apply(lambda row: isholiday(row), axis = 1)
holiday_lookup = holiday_lookup.drop('weekday', axis = 1)


#join with train
train = train.merge(holiday_lookup)

encoder_type = LabelEncoder()
train['type']=encoder_type.fit_transform(train['type'])

In [ ]:
#create shortened data for test purposes
date1 = '2013-01-01'
date2 = '2013-01-03'

#oil_short = oil.loc[oil['date'].between(date1, date2)]
#holidays_short = holidays.loc[holidays['date'].between(date1, date2)]
train_short = train.loc[train['date'].between(date1, date2)].copy()
#transactions_short = transactions.loc[transactions['date'].between(date1, date2)]

train_short

,id,date,store_nbr,family,sales,onpromotion,oilprice,family_id,day,month,weekday,year,type,cluster,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14,0,1,1,1,2013,3,13,2
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14,1,1,1,1,2013,3,13,2
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14,2,1,1,1,2013,3,13,2
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14,3,1,1,1,2013,3,13,2
4,4,2013-01-01,1,BOOKS,0.000,0,93.14,4,1,1,1,2013,3,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5341,5341,2013-01-03,9,POULTRY,509.496,0,92.97,28,3,1,3,2013,1,6,0
5342,5342,2013-01-03,9,PREPARED FOODS,66.000,0,92.97,29,3,1,3,2013,1,6,0
5343,5343,2013-01-03,9,PRODUCE,0.000,0,92.97,30,3,1,3,2013,1,6,0
5344,5344,2013-01-03,9,SCHOOL AND OFFICE SUPPLIES,0.000,0,92.97,31,3,1,3,2013,1,6,0


In [ ]:
test = pd.read_csv('/content/data/test.csv')
test['date'] = test['date'].apply(pd.to_datetime)

# preprocess test data
test = test.merge(oil)
test = test.rename(columns={"dcoilwtico": "oilprice"})



#change family names & type to numeric values
test['family_id']=encoder_family.fit_transform(test['family'])



#split up date into multiple informations
test['day'] = test['date'].apply(lambda time: time.day)
test['month'] = test['date'].apply(lambda time: time.month)
test['weekday'] = test['date'].apply(lambda time: time.dayofweek)
test['year'] = test['date'].apply(lambda time: time.year)

In [ ]:
#test holiday handling

#holiday handling
def isholiday(row):
  #data
  date = row['date']
  event = holidays.loc[holidays['date'] == date][0:1]
  id = row['store_nbr']
  city = stores.loc[stores['store_nbr'] == id]['city'].values
  state = stores.loc[stores['store_nbr'] == id]['state'].values
  
  
  #check if events apply:
  if len(event) > 0:
    national = event['locale'].values == 'National'
    regional = event['locale'].values == 'Regional' and event['locale_name'].values == state
    local = event['locale'].values == 'Local' and event['locale_name'].values == city
    
    if national[0] or regional[0] or local[0]:
      if event['type'].values == 'Holiday' and  event['transferred'].values == False:
        return 2
      elif event['type'].values == 'Transfer':
        return 2
      elif event['type'].values == 'Bridge':
        return 1
      elif event['type'].values == 'Work Day':
        return 0

  #otherwise: check if weekend
  if row['weekday']< 5:
    return 0
  else: 
    return 1


#lookup table  (adds dates to stores table and gets holiday type)
date1, date2 = test['date'].min(), test['date'].max()
holiday_lookup = stores[['store_nbr', 'type', 'cluster']].copy()
holiday_lookup.loc[:,'holiday'] = 0

holiday_lookup = pd.merge(holiday_lookup, pd.DataFrame({'date': pd.date_range(date1, date2, freq = 'd')}), how = "cross")
holiday_lookup['weekday'] = holiday_lookup['date'].apply(lambda time: time.dayofweek)
holiday_lookup.loc[:, 'holiday'] = holiday_lookup.apply(lambda row: isholiday(row), axis = 1)
holiday_lookup = holiday_lookup.drop('weekday', axis = 1)


#join with train
test = test.merge(holiday_lookup)
test['type']=encoder_type.fit_transform(test['type'])

## Test with Shortened Train set

In [ ]:
#create testset only considering automotives at store 1
'''train_auto = train.copy()
print(train_auto.shape[0])
train_auto = train_auto.loc[train_auto['family'] == "AUTOMOTIVE"]
train_auto = train_auto.loc[train_auto['store_nbr'] == 1]
print(train_auto.shape[0])

train_auto['date'] = train_auto['date'].apply(pd.to_datetime)
train_auto['weekday'] = train_auto['date'].apply(lambda time: time.dayofweek)
train_auto['day'] = train_auto['date'].apply(lambda time: time.day)



X_train = train_auto[['id', 'day', 'weekday', 'onpromotion']]
plot_dates = train_auto[['date']]
Y_train = train_auto[['sales']]


train_auto = test.copy()
print(train_auto.shape[0])
train_auto = train_auto.loc[train_auto['family'] == "AUTOMOTIVE"]
train_auto = train_auto.loc[train_auto['store_nbr'] == 1]
print(train_auto.shape[0])

train_auto['date'] = train_auto['date'].apply(pd.to_datetime)
train_auto['weekday'] = train_auto['date'].apply(lambda time: time.dayofweek)
train_auto['day'] = train_auto['date'].apply(lambda time: time.day)

X_test = train_auto[['id', 'day', 'weekday', 'onpromotion']]
'''

'train_auto = train.copy()\nprint(train_auto.shape[0])\ntrain_auto = train_auto.loc[train_auto[\'family\'] == "AUTOMOTIVE"]\ntrain_auto = train_auto.loc[train_auto[\'store_nbr\'] == 1]\nprint(train_auto.shape[0])\n\ntrain_auto[\'date\'] = train_auto[\'date\'].apply(pd.to_datetime)\ntrain_auto[\'weekday\'] = train_auto[\'date\'].apply(lambda time: time.dayofweek)\ntrain_auto[\'day\'] = train_auto[\'date\'].apply(lambda time: time.day)\n\n\n\nX_train = train_auto[[\'id\', \'day\', \'weekday\', \'onpromotion\']]\nplot_dates = train_auto[[\'date\']]\nY_train = train_auto[[\'sales\']]\n\n\ntrain_auto = test.copy()\nprint(train_auto.shape[0])\ntrain_auto = train_auto.loc[train_auto[\'family\'] == "AUTOMOTIVE"]\ntrain_auto = train_auto.loc[train_auto[\'store_nbr\'] == 1]\nprint(train_auto.shape[0])\n\ntrain_auto[\'date\'] = train_auto[\'date\'].apply(pd.to_datetime)\ntrain_auto[\'weekday\'] = train_auto[\'date\'].apply(lambda time: time.dayofweek)\ntrain_auto[\'day\'] = train_auto[\'date\'].a

In [ ]:
#create test data for one family

#cluster = stores[['store_nbr', 'cluster']].groupby('cluster').count().idxmax().values[0]
#stores_in_cluster = stores.loc[stores['cluster'] == cluster]['store_nbr'].values

train_simple = train.copy() #train.loc[train['store_nbr'].isin(stores_in_cluster)].copy()
train_simple = train_simple.loc[train_simple['family_id'] == 32]


cols = ['store_nbr', 'onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'type', 'cluster']
sales_simple = train_simple[['sales', 'date']].copy()

X_train = train_simple.loc[train_simple['date'].between('2013-01-01', '2017-07-31')].copy()
X_test = train_simple.loc[train_simple['date'].between('2017-08-01', '2017-08-15')].copy()  
Y_train = sales_simple.loc[sales_simple['date'].between('2013-01-01', '2017-07-31')].copy()
Y_test = sales_simple.loc[sales_simple['date'].between('2017-08-01', '2017-08-15')].copy()

Y_train = Y_train['sales']
Y_test = Y_test['sales']

print(X_train.shape, '\n', Y_train.shape)

(85158, 15) 
 (85158,)


## Regression

In [ ]:
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#test simple data
cols = ['store_nbr', 'onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'type', 'cluster']

regr = RandomForestRegressor(n_estimators = 100, n_jobs=-1, random_state=1)
regr.fit(X_train[cols].values, Y_train.values.ravel())

def pred(row):
  X = row[cols].values.reshape(1,-1)
  return regr.predict(X)[0]

predict = X_test[cols].apply(lambda row: pred(row), axis = 1)

print(mean_squared_log_error(Y_test.values, predict, squared=False))


0.6301099555826464


## Plots

In [ ]:
'''

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

#fig, ax = plt.subplots(figsize=(8, 5))
cluster = stores[['store_nbr', 'cluster']].groupby('cluster').count().idxmax().values[0]
stores_in_cluster = stores.loc[stores['cluster'] == cluster]['store_nbr'].values

x =  pd.date_range('2017-08-01', '2017-08-15', freq = 'd')
date_form = DateFormatter("%d.%m.%y")
n = len(stores_in_cluster)

Y = Y_test.loc[Y_test['store_nbr'].isin(stores_in_cluster)].copy()

#bring values in useful shape
y_val = Y.values
y_val = y_val.reshape(-1, n).transpose()
p_val = predict.reshape(-1, n).transpose()


#create subplots
if n % 2 == 0:
      f, axs = plt.subplots(n/2, 2, figsize=(10,5))
else:
    f, axs = plt.subplots(int((n+1)/2), 2,  figsize=(10,5),)

#add values
for i in range(n):
    y1 = p_val[i]
    y2 = y_val[i]

    if i % 2 == 0:
      nb = int(i/2)
    else:
      nb = int((i)/2)
    
    axs[nb, i%2].plot(x, y1)
    axs[nb, i%2].plot(x, y2)
    axs[nb, i%2].xaxis.set_major_formatter(date_form)
    axs[nb, i%2].xaxis.set_major_locator(mdates.DayLocator(interval=3))
        
if n % 2 != 0:
    f.delaxes(axs[int((i)/2), 1])

f.tight_layout() 
f.subplots_adjust(bottom=0.1)   ##  Need to play with this number.
f.legend(labels = ['prediction', 'actual sales'], loc=(.7,.1))

#f.suptitle('Results Random Forest Regression')


f.show()

ValueError: ignored

In [ ]:
'''
fig, ax = plt.subplots(figsize=(20, 8))
#get dates
date1, date2 = sales_simple['date'].min(), sales_simple['date'].max()
plot_dates =  pd.date_range(date1, date2, freq = 'd')

leap = []
for each in plot_dates:
    if each.month==12 and each.day ==25:
        leap.append(each)
plot_dates = plot_dates.drop(leap)

s = sales_simple['sales'].values.reshape(n, -1).transpose()
plt.plot(plot_dates, s, label = stores_in_cluster)
date_form = DateFormatter("%m/%y")
ax.xaxis.set_major_formatter(date_form)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
plt.title('total sales')
'''

'\nfig, ax = plt.subplots(figsize=(20, 8))\n#get dates\ndate1, date2 = sales_simple[\'date\'].min(), sales_simple[\'date\'].max()\nplot_dates =  pd.date_range(date1, date2, freq = \'d\')\n\nleap = []\nfor each in plot_dates:\n    if each.month==12 and each.day ==25:\n        leap.append(each)\nplot_dates = plot_dates.drop(leap)\n\ns = sales_simple[\'sales\'].values.reshape(n, -1).transpose()\nplt.plot(plot_dates, s, label = stores_in_cluster)\ndate_form = DateFormatter("%m/%y")\nax.xaxis.set_major_formatter(date_form)\nax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))\nplt.title(\'total sales\')\n'

In [ ]:
#plot_dates.difference(sales_simple['date'])


In [ ]:
'''

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

days = np.zeros(7)
counter  = np.zeros(7)

dates = plot_dates.copy()
dates['weekday'] = dates['date'].apply(lambda time: time.dayofweek)

wd = dates['weekday'].values.flatten()

sales = Y_train.values.flatten()
for i in range(len(plot_dates)):
  weekday = wd[i]
  days[weekday] += sales[i]
  counter[weekday] += 1

days = days/counter
fig, ax = plt.subplots(figsize=(8, 5))
#plt.plot(train_short[['sales']], label = "correct value")
plt.plot(days, label = "prediction")
plt.title('average week')
plt.show()

'''

'\n\nimport matplotlib.pyplot as plt\nfrom matplotlib.dates import DateFormatter\nimport matplotlib.dates as mdates\n\ndays = np.zeros(7)\ncounter  = np.zeros(7)\n\ndates = plot_dates.copy()\ndates[\'weekday\'] = dates[\'date\'].apply(lambda time: time.dayofweek)\n\nwd = dates[\'weekday\'].values.flatten()\n\nsales = Y_train.values.flatten()\nfor i in range(len(plot_dates)):\n  weekday = wd[i]\n  days[weekday] += sales[i]\n  counter[weekday] += 1\n\ndays = days/counter\nfig, ax = plt.subplots(figsize=(8, 5))\n#plt.plot(train_short[[\'sales\']], label = "correct value")\nplt.plot(days, label = "prediction")\nplt.title(\'average week\')\nplt.show()\n\n'

In [ ]:

'''import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

days = np.zeros(31)
counter  = np.zeros(31)

dates = plot_dates.copy()
dates['day'] = dates['date'].apply(lambda time: time.day)

wd = dates['day'].values.flatten()

sales = Y_train.values.flatten()
for i in range(len(plot_dates)):
  weekday = wd[i] - 1
  days[weekday] += sales[i]
  counter[weekday] += 1

days = days/counter
fig, ax = plt.subplots(figsize=(8, 5))
#plt.plot(train_short[['sales']], label = "correct value")
plt.plot(range(1, 32), days, label = "prediction")
plt.title('average month')
plt.show()'''

'import matplotlib.pyplot as plt\nfrom matplotlib.dates import DateFormatter\nimport matplotlib.dates as mdates\n\ndays = np.zeros(31)\ncounter  = np.zeros(31)\n\ndates = plot_dates.copy()\ndates[\'day\'] = dates[\'date\'].apply(lambda time: time.day)\n\nwd = dates[\'day\'].values.flatten()\n\nsales = Y_train.values.flatten()\nfor i in range(len(plot_dates)):\n  weekday = wd[i] - 1\n  days[weekday] += sales[i]\n  counter[weekday] += 1\n\ndays = days/counter\nfig, ax = plt.subplots(figsize=(8, 5))\n#plt.plot(train_short[[\'sales\']], label = "correct value")\nplt.plot(range(1, 32), days, label = "prediction")\nplt.title(\'average month\')\nplt.show()'

## Testing Stuff

In [ ]:
train

,id,date,store_nbr,family,sales,onpromotion,oilprice,family_id,day,month,weekday,year,type,cluster,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14,0,1,1,1,2013,3,13,2
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14,1,1,1,1,2013,3,13,2
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14,2,1,1,1,2013,3,13,2
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14,3,1,1,1,2013,3,13,2
4,4,2013-01-01,1,BOOKS,0.000,0,93.14,4,1,1,1,2013,3,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836939,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,28,15,8,1,2017,1,6,0
2836940,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,29,15,8,1,2017,1,6,0
2836941,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,30,15,8,1,2017,1,6,0
2836942,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,31,15,8,1,2017,1,6,0


In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
#create train&test data
X_train = train.loc[train['date'].between('2013-01-01', '2017-07-31')].copy()
X_test = train.loc[train['date'].between('2017-08-01', '2017-08-15')].copy()
Y_train = train.loc[train['date'].between('2013-01-01', '2017-07-31')]['sales'].copy()
Y_test = train.loc[train['date'].between('2017-08-01', '2017-08-15')]['sales'].copy()

In [ ]:
#test total 

cols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'cluster', 'family_id']

X_test['sales'] = 0

  
regr = RandomForestRegressor(n_estimators = 50, n_jobs=-1, random_state=1)
X = X_train.loc[:,cols].values
Y = Y_train.values.ravel()
regr.fit(X, Y)
pred = regr.predict(X_test[cols].values)

X_test.loc[:,'sales'] = pred



print('total: ', mean_squared_log_error(Y_test.values, X_test['sales'].values, squared=False))


total:  0.5686385549148442


In [ ]:
#test with cluster
'''
cluster_nbr = train['cluster'].max()
cols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'family_id']

X_test['sales'] = 0



for fam in range(1, cluster_nbr):



  idx_train = X_train['cluster']== fam
  regr = RandomForestRegressor(n_estimators = 50, n_jobs=-1, random_state=1)
  X = X_train.loc[idx_train][cols].values
  Y = Y_train.loc[idx_train].values.ravel()
  regr.fit(X, Y)
  idx_test = X_test['cluster']== fam
  pred = regr.predict(X_test[idx_test][cols].values)

  X_test.loc[idx_test,'sales'] = pred
  print('group ', fam, ': ', mean_squared_log_error(Y_test[idx_test].values, pred, squared=False))



print('total: ', mean_squared_log_error(Y_test.values, X_test['sales'].values, squared=False))
'''


"\ncluster_nbr = train['cluster'].max()\ncols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'family_id']\n\nX_test['sales'] = 0\n\n\n\nfor fam in range(1, cluster_nbr):\n\n\n\n  idx_train = X_train['cluster']== fam\n  regr = RandomForestRegressor(n_estimators = 50, n_jobs=-1, random_state=1)\n  X = X_train.loc[idx_train][cols].values\n  Y = Y_train.loc[idx_train].values.ravel()\n  regr.fit(X, Y)\n  idx_test = X_test['cluster']== fam\n  pred = regr.predict(X_test[idx_test][cols].values)\n\n  X_test.loc[idx_test,'sales'] = pred\n  print('group ', fam, ': ', mean_squared_log_error(Y_test[idx_test].values, pred, squared=False))\n\n\n\nprint('total: ', mean_squared_log_error(Y_test.values, X_test['sales'].values, squared=False))\n"

In [ ]:
#test with family
'''
family_nbr = train['family_id'].max()
cols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'cluster']

X_test['sales'] = 0



for fam in range(family_nbr):



  idx_train = X_train['family_id']== fam
  regr = RandomForestRegressor(n_estimators = 50, n_jobs=-1, random_state=1)
  X = X_train.loc[idx_train][cols].values
  Y = Y_train.loc[idx_train].values.ravel()
  regr.fit(X, Y)
  idx_test = X_test['family_id']== fam
  pred = regr.predict(X_test[idx_test][cols].values)

  X_test.loc[idx_test,'sales'] = pred
  print('group ', fam, ': ', mean_squared_log_error(Y_test[idx_test].values, pred, squared=False))



print('total: ', mean_squared_log_error(Y_test.values, X_test['sales'].values, squared=False))
'''

"\nfamily_nbr = train['family_id'].max()\ncols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'cluster']\n\nX_test['sales'] = 0\n\n\n\nfor fam in range(family_nbr):\n\n\n\n  idx_train = X_train['family_id']== fam\n  regr = RandomForestRegressor(n_estimators = 50, n_jobs=-1, random_state=1)\n  X = X_train.loc[idx_train][cols].values\n  Y = Y_train.loc[idx_train].values.ravel()\n  regr.fit(X, Y)\n  idx_test = X_test['family_id']== fam\n  pred = regr.predict(X_test[idx_test][cols].values)\n\n  X_test.loc[idx_test,'sales'] = pred\n  print('group ', fam, ': ', mean_squared_log_error(Y_test[idx_test].values, pred, squared=False))\n\n\n\nprint('total: ', mean_squared_log_error(Y_test.values, X_test['sales'].values, squared=False))\n"

In [ ]:
#test Ridge
from sklearn.linear_model import Ridge


cols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'cluster', 'family_id']

X_test['sales'] = 0

  
clf = Ridge()
X = X_train.loc[:,cols]
Y = Y_train.values.ravel()
clf.fit(X, Y)
pred = regr.predict(X_test[cols].values)

X_test.loc[:,'sales'] = pred



print('total: ', mean_squared_log_error(Y_test.values, X_test['sales'].values, squared=False))


total:  0.568638554914844


## CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
idx_train = (X_train['family_id']== 1)
X = X_train.loc[idx_train][cols].values
Y = Y_train.loc[idx_train].values.ravel()

In [ ]:
cols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'cluster']
n_features = len(cols)

model = keras.Sequential()
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation = 'softplus'))
model.compile(optimizer='adam', loss="mean_squared_logarithmic_error")

In [ ]:
model.fit(X, Y, epochs=100, verbose=0)
idx_test = (X_test['family_id']== 1) 

pred = model.predict(X_test[idx_test][cols].values)

print('result: ', "{:10.8f}".format(mean_squared_log_error(Y_test[idx_test].values, pred, squared=False)))


group  13 :  0.29160563


In [ ]:
family_nbr = train['family_id'].max()
cols = ['onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'cluster']

X_test['sales'] = 0


for fam in range(family_nbr):

  idx_train = X_train['family_id']== fam
  X = X_train.loc[idx_train][cols].values
  Y = Y_train.loc[idx_train].values.ravel()
  idx_test = X_test['family_id']== fam


  model.fit(X, Y, epochs=100, verbose=0)
  pred = model.predict(X_test[idx_test][cols].values)

  X_test.loc[idx_test,'sales'] = pred
  print('group ', fam, ': ', mean_squared_log_error(Y_test[idx_test].values, pred, squared=False))



print('total: ', mean_squared_log_error(Y_test.values, X_test['sales'].values, squared=False))


group  0 :  0.6999504877511238
group  1 :  0.29448963454606886
group  2 :  0.7353078073442026
group  3 :  0.6516831807016812
group  4 :  0.07105611627197571
group  5 :  0.8590270307853939
group  6 :  0.7198398120486363
group  7 :  0.6474041448786101
group  8 :  0.578944781700307
group  9 :  0.492750025455109
group  10 :  0.8658886382111128
group  11 :  0.6629999488506331
group  12 :  0.5799241883042292
group  13 :  1.0027131915350938
group  14 :  0.6088994577406944
group  15 :  0.7418512612301199
group  16 :  0.6507070455192211
group  17 :  0.2989005509141689
group  18 :  0.5630722856566412
group  19 :  1.3431410873703498
group  20 :  1.420920594950977
group  21 :  0.8531524993649259
group  22 :  0.7799688171975021
group  23 :  1.1188969931272061
group  24 :  0.824818607648025
group  25 :  0.5737600569117739
group  26 :  0.9641034038030398
group  27 :  0.8309055903889749
group  28 :  0.8240629642742751
group  29 :  0.7249547516005722
group  30 :  0.7766139653218657
group  31 :  0.74785

In [ ]:
X_test

,id,date,store_nbr,family,sales,onpromotion,oilprice,family_id,day,month,weekday,year,type,cluster,holiday
2810214,2974158,2017-08-01,1,AUTOMOTIVE,0,0,49.19,0,1,8,1,2017,3,13,0
2810215,2974159,2017-08-01,1,BABY CARE,0,0,49.19,1,1,8,1,2017,3,13,0
2810216,2974160,2017-08-01,1,BEAUTY,0,0,49.19,2,1,8,1,2017,3,13,0
2810217,2974161,2017-08-01,1,BEVERAGES,0,26,49.19,3,1,8,1,2017,3,13,0
2810218,2974162,2017-08-01,1,BOOKS,0,0,49.19,4,1,8,1,2017,3,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836939,3000883,2017-08-15,9,POULTRY,0,0,47.57,28,15,8,1,2017,1,6,0
2836940,3000884,2017-08-15,9,PREPARED FOODS,0,1,47.57,29,15,8,1,2017,1,6,0
2836941,3000885,2017-08-15,9,PRODUCE,0,148,47.57,30,15,8,1,2017,1,6,0
2836942,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,0,8,47.57,31,15,8,1,2017,1,6,0


## Full Dataset

In [ ]:
'''models = []


family_nbr = train['family_id'].max()

for fam in range(family_nbr):
  train_fam = train.loc[train['family_id']== fam]
  X_train = train_fam[['store_nbr', 'onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month', 'type', 'cluster']].values
  Y_train = train_fam['sales']

  regr = RandomForestRegressor(max_depth=2, random_state=0)
  regr.fit(X_train, Y_train.values.ravel())

  models.append(regr)



In [ ]:
'''def pred(row):
  i = row['family_id']
  X = row[['store_nbr', 'onpromotion', 'oilprice', 'holiday', 'weekday', 'day', 'month']].values.reshape(1,-1)
  return models[i-1].predict(X)[0]

test['prediction']  = test.apply(lambda row: pred(row), axis = 1)



In [ ]:
'''path = '/content/submission.csv'
test[['id', 'sales']].to_csv(path, header = 'id,sales', index = False)